<a href="https://colab.research.google.com/github/Arif111866/Neural-Network-/blob/main/assignment_1_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import necessary modules

In [1]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model

# Load Dataset

In [2]:
# Load data
(trainX, trainY), (testX, testY) = cifar10.load_data()

# Investigate loaded data
print('trainX.shape: {}, trainY.shape: {}, testX.shape: {}, testY.shape: {})'.format(trainX.shape, trainY.shape, testX.shape, testY.shape))
print('trainX.dtype: {}, trainY.dtype: {}, testX.dtype: {}, testY.dtype: {}'.format(trainX.dtype, trainY.dtype, testX.dtype, testY.dtype))
print('trainX.Range: {} - {}, testX.Range: {} - {}'.format(trainX.max(), trainX.min(), testX.max(), testX.min()))


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
trainX.shape: (50000, 32, 32, 3), trainY.shape: (50000, 1), testX.shape: (10000, 32, 32, 3), testY.shape: (10000, 1))
trainX.dtype: uint8, trainY.dtype: uint8, testX.dtype: uint8, testY.dtype: uint8
trainX.Range: 255 - 0, testX.Range: 255 - 0


# Prepare datasets

In [3]:
# Turn 2D images into 3D so that trainX and TrainY will be 4D since Convolutional layer takes 4D data as input.
trainX = np.expand_dims(trainX, axis = -1)
testX = np.expand_dims(testX, axis = -1)

# Investigate updated x
print('trainX.shape: {}, testX.shape: {})'.format(trainX.shape, testX.shape))
print('trainX.dtype: {}, testX.dtype: {}'.format(trainX.dtype, testX.dtype))
print('trainX.Range: {} - {}, testX.Range: {} - {}'.format(trainX.max(), trainX.min(), testX.max(), testX.min()))

# Turn y into one-hot-encoding, so that we can use 10 neurons in the output layer
trainY = to_categorical(trainY, num_classes = 10)
testY = to_categorical(testY, num_classes = 10)

# Investigate updated y
print('trainY.shape: {}, testY.shape: {})'.format(trainY.shape, testY.shape))
print('trainY.dtype: {}, testX.dtype: {}'.format(trainY.dtype, testY.dtype))
print(trainY[:5])

trainX.shape: (50000, 32, 32, 3, 1), testX.shape: (10000, 32, 32, 3, 1))
trainX.dtype: uint8, testX.dtype: uint8
trainX.Range: 255 - 0, testX.Range: 255 - 0
trainY.shape: (50000, 10), testY.shape: (10000, 10))
trainY.dtype: float64, testX.dtype: float64
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]


# Build a fully connected neural network

In [4]:
mobileNet = MobileNet(input_shape = (32, 32, 3), include_top = False)
# mobileNet.trainable = False
mobileNet.summary(show_trainable= True)

<ipython-input-4-f74662fa89e4>:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  mobileNet = MobileNet(input_shape = (32, 32, 3), include_top = False)


17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "mobilenet_1.00_224"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Layer (type)                        ┃ Output Shape                 ┃       Param # ┃ Traina… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ input_layer (InputLayer)            │ (None, 32, 32, 3)            │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv1 (Conv2D)                      │ (None, 16, 16, 32)           │           864 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv1_bn (BatchNormalization)       │ (None, 16, 16, 32)           │           128 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv1_relu (ReLU)                   │ (None, 16, 16, 32)           │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_1 (DepthwiseConv2D)         │ (None, 16, 16, 32)           │           288 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_1_bn (BatchNormalization)   │ (None, 16, 16, 32)           │           128 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_1_relu (ReLU)               │ (None, 16, 16, 32)           │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_1 (Conv2D)                  │ (None, 16, 16, 64)           │         2,048 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_1_bn (BatchNormalization)   │ (None, 16, 16, 64)           │           256 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_1_relu (ReLU)               │ (None, 16, 16, 64)           │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pad_2 (ZeroPadding2D)          │ (None, 17, 17, 64)           │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_2 (DepthwiseConv2D)         │ (None, 8, 8, 64)             │           576 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_2_bn (BatchNormalization)   │ (None, 8, 8, 64)             │           256 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_2_relu (ReLU)               │ (None, 8, 8, 64)             │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_2 (Conv2D)                  │ (None, 8, 8, 128)            │         8,192 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_2_bn (BatchNormalization)   │ (None, 8, 8, 128)            │           512 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_2_relu (ReLU)               │ (None, 8, 8, 128)            │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_3 (DepthwiseConv2D)         │ (None, 8, 8, 128)            │         1,152 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_3_bn (BatchNormalization)   │ (None, 8, 8, 128)            │           512 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_3_relu (ReLU)

 Total params: 3,228,864 (12.32 MB)

 Trainable params: 3,206,976 (12.23 MB)

 Non-trainable params: 21,888 (85.50 KB)

In [5]:
inputs = mobileNet.input
x = mobileNet.output
x = Flatten()(x)
x = Dense(23, activation = 'relu')(x)
outputs = Dense(10, name = 'OutputLayer', activation = 'softmax')(x)
model = Model(inputs, outputs, name = 'Multi-Class-Classifier')
model.summary(show_trainable= True)

Model: "Multi-Class-Classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Layer (type)                        ┃ Output Shape                 ┃       Param # ┃ Traina… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ input_layer (InputLayer)            │ (None, 32, 32, 3)            │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv1 (Conv2D)                      │ (None, 16, 16, 32)           │           864 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv1_bn (BatchNormalization)       │ (None, 16, 16, 32)           │           128 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv1_relu (ReLU)                   │ (None, 16, 16, 32)           │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_1 (DepthwiseConv2D)         │ (None, 16, 16, 32)           │           288 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_1_bn (BatchNormalization)   │ (None, 16, 16, 32)           │           128 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_1_relu (ReLU)               │ (None, 16, 16, 32)           │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_1 (Conv2D)                  │ (None, 16, 16, 64)           │         2,048 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_1_bn (BatchNormalization)   │ (None, 16, 16, 64)           │           256 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_1_relu (ReLU)               │ (None, 16, 16, 64)           │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pad_2 (ZeroPadding2D)          │ (None, 17, 17, 64)           │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_2 (DepthwiseConv2D)         │ (None, 8, 8, 64)             │           576 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_2_bn (BatchNormalization)   │ (None, 8, 8, 64)             │           256 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_2_relu (ReLU)               │ (None, 8, 8, 64)             │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_2 (Conv2D)                  │ (None, 8, 8, 128)            │         8,192 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_2_bn (BatchNormalization)   │ (None, 8, 8, 128)            │           512 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_2_relu (ReLU)               │ (None, 8, 8, 128)            │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_3 (DepthwiseConv2D)         │ (None, 8, 8, 128)            │         1,152 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_3_bn (BatchNormalization)   │ (None, 8, 8, 128)            │           512 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_3_relu (ReLU)

 Total params: 3,252,679 (12.41 MB)

 Trainable params: 3,230,791 (12.32 MB)

 Non-trainable params: 21,888 (85.50 KB)

In [6]:
for layer in model.layers[: - 4]:
  layer.trainable = False
model.summary(show_trainable = True)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(trainX, trainY, validation_split = 0.1, epochs = 10)


Model: "Multi-Class-Classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Layer (type)                        ┃ Output Shape                 ┃       Param # ┃ Traina… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ input_layer (InputLayer)            │ (None, 32, 32, 3)            │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv1 (Conv2D)                      │ (None, 16, 16, 32)           │           864 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv1_bn (BatchNormalization)       │ (None, 16, 16, 32)           │           128 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv1_relu (ReLU)                   │ (None, 16, 16, 32)           │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_1 (DepthwiseConv2D)         │ (None, 16, 16, 32)           │           288 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_1_bn (BatchNormalization)   │ (None, 16, 16, 32)           │           128 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_1_relu (ReLU)               │ (None, 16, 16, 32)           │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_1 (Conv2D)                  │ (None, 16, 16, 64)           │         2,048 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_1_bn (BatchNormalization)   │ (None, 16, 16, 64)           │           256 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_1_relu (ReLU)               │ (None, 16, 16, 64)           │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pad_2 (ZeroPadding2D)          │ (None, 17, 17, 64)           │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_2 (DepthwiseConv2D)         │ (None, 8, 8, 64)             │           576 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_2_bn (BatchNormalization)   │ (None, 8, 8, 64)             │           256 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_2_relu (ReLU)               │ (None, 8, 8, 64)             │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_2 (Conv2D)                  │ (None, 8, 8, 128)            │         8,192 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_2_bn (BatchNormalization)   │ (None, 8, 8, 128)            │           512 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_2_relu (ReLU)               │ (None, 8, 8, 128)            │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_3 (DepthwiseConv2D)         │ (None, 8, 8, 128)            │         1,152 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_3_bn (BatchNormalization)   │ (None, 8, 8, 128)            │           512 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_3_relu (ReLU)

 Total params: 3,252,679 (12.41 MB)

 Trainable params: 23,815 (93.03 KB)

 Non-trainable params: 3,228,864 (12.32 MB)

Epoch 1/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - accuracy: 0.1652 - loss: 2.2330 - val_accuracy: 0.2040 - val_loss: 2.1550
Epoch 2/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.1984 - loss: 2.1622 - val_accuracy: 0.2034 - val_loss: 2.1456
Epoch 3/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.2034 - loss: 2.1573 - val_accuracy: 0.2094 - val_loss: 2.1399
Epoch 4/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.2047 - loss: 2.1484 - val_accuracy: 0.2054 - val_loss: 2.1364
Epoch 5/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.2088 - loss: 2.1449 - val_accuracy: 0.2040 - val_loss: 2.1332
Epoch 6/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.2103 - loss: 2.1430 - val_accuracy: 0.2116 - val_loss: 2.1317
Epoch 7/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.2138 - loss: 2.1408 - val_accuracy: 0.2076 - val_loss: 2.1317
Epoch 8/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.2078 - loss: 2.1415 

In [7]:
# Unfreez weights of some part of feature extractor
for layer in model.layers[-6:-4]:
  layer.trainable = True
model.summary(show_trainable = True)

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(trainX, trainY, validation_split = 0.1, epochs = 10)

Model: "Multi-Class-Classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Layer (type)                        ┃ Output Shape                 ┃       Param # ┃ Traina… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ input_layer (InputLayer)            │ (None, 32, 32, 3)            │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv1 (Conv2D)                      │ (None, 16, 16, 32)           │           864 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv1_bn (BatchNormalization)       │ (None, 16, 16, 32)           │           128 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv1_relu (ReLU)                   │ (None, 16, 16, 32)           │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_1 (DepthwiseConv2D)         │ (None, 16, 16, 32)           │           288 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_1_bn (BatchNormalization)   │ (None, 16, 16, 32)           │           128 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_1_relu (ReLU)               │ (None, 16, 16, 32)           │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_1 (Conv2D)                  │ (None, 16, 16, 64)           │         2,048 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_1_bn (BatchNormalization)   │ (None, 16, 16, 64)           │           256 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_1_relu (ReLU)               │ (None, 16, 16, 64)           │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pad_2 (ZeroPadding2D)          │ (None, 17, 17, 64)           │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_2 (DepthwiseConv2D)         │ (None, 8, 8, 64)             │           576 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_2_bn (BatchNormalization)   │ (None, 8, 8, 64)             │           256 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_2_relu (ReLU)               │ (None, 8, 8, 64)             │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_2 (Conv2D)                  │ (None, 8, 8, 128)            │         8,192 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_2_bn (BatchNormalization)   │ (None, 8, 8, 128)            │           512 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_2_relu (ReLU)               │ (None, 8, 8, 128)            │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_3 (DepthwiseConv2D)         │ (None, 8, 8, 128)            │         1,152 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_3_bn (BatchNormalization)   │ (None, 8, 8, 128)            │           512 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_3_relu (ReLU)

 Total params: 3,300,311 (12.59 MB)

 Trainable params: 1,074,439 (4.10 MB)

 Non-trainable params: 2,178,240 (8.31 MB)

 Optimizer params: 47,632 (186.07 KB)

Epoch 1/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.2066 - loss: 2.1356 - val_accuracy: 0.2128 - val_loss: 2.1268
Epoch 2/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.2139 - loss: 2.1290 - val_accuracy: 0.2040 - val_loss: 2.1297
Epoch 3/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.2112 - loss: 2.1305 - val_accuracy: 0.2138 - val_loss: 2.1241
Epoch 4/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.2122 - loss: 2.1338 - val_accuracy: 0.2054 - val_loss: 2.1235
Epoch 5/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.2143 - loss: 2.1283 - val_accuracy: 0.2134 - val_loss: 2.1232
Epoch 6/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.2163 - loss: 2.1240 - val_accuracy: 0.2114 - val_loss: 2.1256
Epoch 7/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.2136 - loss: 2.1269 - val_accuracy: 0.2082 - val_loss: 2.1269
Epoch 8/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.2093 - loss: 2.1315 

In [8]:
# Unfreez weights of more layers of feature extractor
for layer in model.layers[-9:-6]:
  layer.trainable = True
model.summary(show_trainable = True)

# Retrain model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(trainX, trainY, validation_split = 0.1, epochs = 10)


Model: "Multi-Class-Classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Layer (type)                        ┃ Output Shape                 ┃       Param # ┃ Traina… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ input_layer (InputLayer)            │ (None, 32, 32, 3)            │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv1 (Conv2D)                      │ (None, 16, 16, 32)           │           864 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv1_bn (BatchNormalization)       │ (None, 16, 16, 32)           │           128 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv1_relu (ReLU)                   │ (None, 16, 16, 32)           │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_1 (DepthwiseConv2D)         │ (None, 16, 16, 32)           │           288 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_1_bn (BatchNormalization)   │ (None, 16, 16, 32)           │           128 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_1_relu (ReLU)               │ (None, 16, 16, 32)           │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_1 (Conv2D)                  │ (None, 16, 16, 64)           │         2,048 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_1_bn (BatchNormalization)   │ (None, 16, 16, 64)           │           256 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_1_relu (ReLU)               │ (None, 16, 16, 64)           │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pad_2 (ZeroPadding2D)          │ (None, 17, 17, 64)           │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_2 (DepthwiseConv2D)         │ (None, 8, 8, 64)             │           576 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_2_bn (BatchNormalization)   │ (None, 8, 8, 64)             │           256 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_2_relu (ReLU)               │ (None, 8, 8, 64)             │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_2 (Conv2D)                  │ (None, 8, 8, 128)            │         8,192 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_2_bn (BatchNormalization)   │ (None, 8, 8, 128)            │           512 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_2_relu (ReLU)               │ (None, 8, 8, 128)            │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_3 (DepthwiseConv2D)         │ (None, 8, 8, 128)            │         1,152 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_3_bn (BatchNormalization)   │ (None, 8, 8, 128)            │           512 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_3_relu (ReLU)

 Total params: 3,300,311 (12.59 MB)

 Trainable params: 1,085,703 (4.14 MB)

 Non-trainable params: 2,166,976 (8.27 MB)

 Optimizer params: 47,632 (186.07 KB)

Epoch 1/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.1766 - loss: 3.2832 - val_accuracy: 0.2162 - val_loss: 2.0641
Epoch 2/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.2456 - loss: 2.0057 - val_accuracy: 0.2828 - val_loss: 1.9455
Epoch 3/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.2894 - loss: 1.8944 - val_accuracy: 0.3118 - val_loss: 1.8702
Epoch 4/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.3258 - loss: 1.7999 - val_accuracy: 0.3466 - val_loss: 1.8056
Epoch 5/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.3625 - loss: 1.7106 - val_accuracy: 0.3508 - val_loss: 1.7707
Epoch 6/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.3911 - loss: 1.6345 - val_accuracy: 0.3806 - val_loss: 1.7173
Epoch 7/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.4279 - loss: 1.5496 - val_accuracy: 0.3886 - val_loss: 1.7368
Epoch 8/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.4561 - loss: 1.474

In [9]:
# Unfreez weights of more layers of feature extractor
for layer in model.layers:
  layer.trainable = True
model.summary(show_trainable = True)

# Retrain model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(trainX, trainY, validation_split = 0.1, epochs = 10)


Model: "Multi-Class-Classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Layer (type)                        ┃ Output Shape                 ┃       Param # ┃ Traina… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ input_layer (InputLayer)            │ (None, 32, 32, 3)            │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv1 (Conv2D)                      │ (None, 16, 16, 32)           │           864 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv1_bn (BatchNormalization)       │ (None, 16, 16, 32)           │           128 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv1_relu (ReLU)                   │ (None, 16, 16, 32)           │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_1 (DepthwiseConv2D)         │ (None, 16, 16, 32)           │           288 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_1_bn (BatchNormalization)   │ (None, 16, 16, 32)           │           128 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_1_relu (ReLU)               │ (None, 16, 16, 32)           │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_1 (Conv2D)                  │ (None, 16, 16, 64)           │         2,048 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_1_bn (BatchNormalization)   │ (None, 16, 16, 64)           │           256 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_1_relu (ReLU)               │ (None, 16, 16, 64)           │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pad_2 (ZeroPadding2D)          │ (None, 17, 17, 64)           │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_2 (DepthwiseConv2D)         │ (None, 8, 8, 64)             │           576 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_2_bn (BatchNormalization)   │ (None, 8, 8, 64)             │           256 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_2_relu (ReLU)               │ (None, 8, 8, 64)             │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_2 (Conv2D)                  │ (None, 8, 8, 128)            │         8,192 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_2_bn (BatchNormalization)   │ (None, 8, 8, 128)            │           512 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_pw_2_relu (ReLU)               │ (None, 8, 8, 128)            │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_3 (DepthwiseConv2D)         │ (None, 8, 8, 128)            │         1,152 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_3_bn (BatchNormalization)   │ (None, 8, 8, 128)            │           512 │    Y    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ conv_dw_3_relu (ReLU)

 Total params: 5,424,087 (20.69 MB)

 Trainable params: 3,230,791 (12.32 MB)

 Non-trainable params: 21,888 (85.50 KB)

 Optimizer params: 2,171,408 (8.28 MB)

Epoch 1/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 44s 16ms/step - accuracy: 0.4983 - loss: 1.4736 - val_accuracy: 0.7108 - val_loss: 0.8583
Epoch 2/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - accuracy: 0.7037 - loss: 0.8911 - val_accuracy: 0.7590 - val_loss: 0.7334
Epoch 3/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - accuracy: 0.7458 - loss: 0.7661 - val_accuracy: 0.7400 - val_loss: 0.7901
Epoch 4/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.7716 - loss: 0.6851 - val_accuracy: 0.7566 - val_loss: 0.7354
Epoch 5/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.8011 - loss: 0.5990 - val_accuracy: 0.7578 - val_loss: 0.7390
Epoch 6/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.8164 - loss: 0.5671 - val_accuracy: 0.8044 - val_loss: 0.6205
Epoch 7/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.8376 - loss: 0.4953 - val_accuracy: 0.8062 - val_loss: 0.5985
Epoch 8/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - accuracy: 0.8543 - loss: 

In [10]:
# Evaluate model performance
model.evaluate(testX, testY)

# Predict Y values
predictY = model.predict(testX)

print('OriginalY   PredictedY')
print('=========   ==========')
for i in range(10):
  print(np.argmax(testY[i]), '\t\t', np.argmax(predictY[i]))

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.7766 - loss: 0.7847
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step
OriginalY   PredictedY
=========   ==========
3 		 3
8 		 8
8 		 8
0 		 0
6 		 6
6 		 6
1 		 1
6 		 6
3 		 3
1 		 1
